In [1]:
import os
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use("bmh")

/Users/danieloliveira/opt/anaconda3/envs/fsl-ts/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Generator(nn.Module):
    def __init__(self, n_input, n_output) -> None:
        super().__init__()
        self.n_input = n_input
        self.n_output = n_output

        self.fc0 = nn.Sequential(
                    nn.Linear(self.n_input, 256),
                    nn.LeakyReLU(0.2)
                    )
        self.fc1 = nn.Sequential(
                    nn.Linear(256, 512),
                    nn.LeakyReLU(0.2)
                    )
        self.fc2 = nn.Sequential(
                    nn.Linear(512, self.n_output),
                    nn.Tanh()
                    )
        
    def forward(self, x):
        x = self.fc0(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x
    
class Discriminator(nn.Module):
    def __init__(self, n_input, n_output) -> None:
        super().__init__()
        self.n_input = n_input
        self.n_output = n_output

        self.fc0 = nn.Sequential(
                    nn.Linear(self.n_input, 256),
                    nn.LeakyReLU(0.2)
                    )
        self.fc1 = nn.Sequential(
                    nn.Linear(256, 512),
                    nn.LeakyReLU(0.2)
                    )
        self.fc2 = nn.Sequential(
                    nn.Linear(512, self.n_output),
                    nn.Sigmoid()
                    )
        
    def forward(self, x):
        x = self.fc0(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

### 0. Load data and filter rows

In [3]:
import pandas as pd
from settings import INPUTS_PATH

data = pd.read_csv(os.path.join(INPUTS_PATH, "adult.csv"))
data = data[data != "?"].dropna()

In [4]:
data.shape

(30162, 15)

In [5]:
data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
5,34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States,<=50K
6,38,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,Male,0,3770,40,United-States,<=50K


In [6]:
data = data.copy()
num_cols = ['age', 'education.num', 'hours.per.week']
cat_cols = []

n_epoch = 10
n_iter = 10
batch_size_perc = 0.2

learning_rate = 2e-4
criterion = nn.BCELoss()

### 1. Preprocess data

In [25]:
import torch
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import numpy as np

num_cols_df = data[num_cols]
cat_cols_df = data[cat_cols]

# scale numerical columns
num_scaler = MinMaxScaler()
num_scaler.fit(data[num_cols])
proc_num_cols_df = num_scaler.transform(data[num_cols])

# scale categorical columns
cat_scaler = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_scaler.fit(data[cat_cols])
proc_cat_cols_df = cat_scaler.transform(data[cat_cols])

if (proc_num_cols_df.shape == 0) and (proc_cat_cols_df.shape == 0):
    raise Exception("No data left in the preprocess step")
elif proc_num_cols_df.shape[1] == 0:
    proc_data = torch.tensor(proc_cat_cols_df.copy()).float()
elif proc_cat_cols_df.shape[1] == 0:
    proc_data = torch.tensor(proc_num_cols_df.copy()).float()
else:
    proc_data = np.concatenate([proc_num_cols_df, proc_cat_cols_df], axis=1).float()


### 2. Train GAN

In [26]:
import torch

def get_random_data_batch(data, batch_size):
    nrows = data.shape[0]

    bathc_idx = np.random.randint(low=0, high=nrows, size=batch_size)
    selected_data = data[bathc_idx, :]

    return torch.tensor(selected_data).float()

In [27]:
# util dims
nrows = proc_data.shape[0]
ncols = proc_data.shape[1]
batch_size = int(nrows * batch_size_perc)

# adversarial ground truths
true_real = torch.ones((batch_size, 1))
true_fake = torch.zeros((batch_size, 1))

# instantiate generator and discriminator
generator = Generator(n_input=proc_data.shape[1], n_output=proc_data.shape[1])
discriminator = Discriminator(n_input=proc_data.shape[1], n_output=1)

# define optimizers
g_optimizer = optim.Adam(generator.parameters(), lr=learning_rate)
d_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate)

## for epoch in range(n_epochs):
##   for i in range(n_iter):

epoch = 0
i = 0

batch_data = get_random_data_batch(data=proc_data, batch_size=batch_size)
noise = torch.normal(mean=0, std=1, size=(batch_size, ncols))

# run generator
gen_data = generator.forward(x=noise)

# run discriminator
prediction_fake = discriminator.forward(x=gen_data)

# compute discriminator loss
d_optimizer.zero_grad()
error_fake = criterion(prediction_fake, true_fake)
error_fake.backward()

predictions_real = discriminator.forward(x=batch_data)
error_real = criterion(predictions_real, true_real)
error_real.backward()

d_optimizer.step()
d_error = error_fake + error_real

# compute generator loss
g_optimizer.zero_grad()
g_error_fake = criterion(prediction_fake, true_real)
g_error_fake.backward()
g_optimizer.step()

/var/folders/y5/gwwq15s92v97s_yvjxy3cqn40000gn/T/ipykernel_53912/733733988.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(selected_data).float()


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [128]:
error_fake

tensor(0.6698, grad_fn=<BinaryCrossEntropyBackward0>)

In [129]:
error_real

tensor(0.6784, grad_fn=<BinaryCrossEntropyBackward0>)